# Cat Herder - Simple Debugger using LLM Reflection

## Galen Wilkerson

One of the most common tasks using LLMs is for programming.

However, for several reasons, LLMs are not perfect programmers (yet).  Very often, you get the code back and it has bugs, either problems with import statements, runtime errors, etc., and you find yourself re-submitting your code with the errors and hoping the LLM gets it right.

This program lets you:

* modify existing code or do something new
    * it asks you how many debug iterations you would like 
    * it then accesss chatGPT to write the program
    * executes the program locally
    * detects if there were any errors
* re-submits the new code with the errors
    * if no errors, it stops iterating

In [1]:
import openai
import os

def read_api_key(file_path):
    """
    Reads the API key from a specified file.

    Parameters:
    file_path (str): The path to the file containing the API key.

    Returns:
    str: The API key.
    """
    with open(file_path, 'r') as file:
        return file.read().strip()

def extract_python_code(response):
    """
    Extracts the Python code from the OpenAI API response.

    Parameters:
    response (dict): The response dictionary from the OpenAI API.

    Returns:
    str: The extracted Python code.
    """
    # Extract the message content
    content = response['choices'][0]['message']['content']

    # Find the start and end of the code block
    start_index = content.find("```python\n") + len("```python\n")
    end_index = content.find("\n```", start_index)

    # Extract the code
    code = content[start_index:end_index]
    
    return code

def call_chatgpt(prompt, client, special_instructions='Implement a python script in a single code block to perform this task: '):
    """
    Calls the OpenAI API to generate code based on the provided prompt.

    Parameters:
    prompt (str): The prompt describing the desired code.
    client (openai.OpenAI): The OpenAI client.

    Returns:
    str: The generated Python code.
    """
    response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": special_instructions + prompt}
        ]
    )
    # print(response)  # Add this line to inspect the response structure
    return extract_python_code(response.dict())

def debug_code_with_chatgpt(original_prompt, code, error, client):
    """
    Calls the OpenAI API to debug the provided code based on the encountered error.

    Parameters:
    original_prompt (str): The original prompt describing the desired code.
    code (str): The code that produced an error.
    error (str): The error message.
    client (openai.OpenAI): The OpenAI client.

    Returns:
    str: The suggested fix for the code.
    """
    prompt = f"The original prompt was:\n\n{original_prompt}\n\nHere is the code:\n\n{code}\n\nIt produced the following error:\n\n{error}\n\nPlease help me fix it."
    return call_chatgpt(prompt, client)

def save_code_to_file(code, file_path='script.py'):
    """
    Saves the provided code to a specified file.

    Parameters:
    code (str): The code to be saved.
    file_path (str): The path to the file where the code should be saved. Default is 'script.py'.
    """
    with open(file_path, 'w') as file:
        file.write(code)

def main():
    """
    Main function to read API key, generate code, and debug it iteratively.
    """
    api_key = read_api_key('api_key.txt')
    openai.api_key = api_key
    client = openai

    choice = input("Do you want to (1) enter a new code prompt or (2) modify existing code? Enter 1 or 2: ")
    
    if choice == '1':
        user_prompt = input("Please enter the code prompt: ")
        code = call_chatgpt(user_prompt, client)
    elif choice == '2':
        file_path = input("Please enter the path to the existing code file: ")
        with open(file_path, 'r') as file:
            code = file.read()
        user_prompt = input("Please describe the modifications you want to make: ")
        code = call_chatgpt(user_prompt, client, special_instructions='Modify the following code:\n\n' + code + '\n\n')
    else:
        print("Invalid choice.")
        return

    iterations = int(input("How many debug iterations should be done? "))

    print('------------------')
    print("Initial code generated by ChatGPT:\n", code)
    print('------------------')

    last_error_message = None

    for i in range(iterations):
        print(f"\nIteration {i+1}:")

        try:
            exec(code)
            print("Code executed successfully.")
            save_code_to_file(code)
            print('Saved to script.py')
            break
        except Exception as e:
            error_message = str(e)
            print("Error encountered:", error_message)

            if error_message == last_error_message:
                print("Encountered the same error. Stopping iterations.")
                break
            last_error_message = error_message
            
            # Get help from ChatGPT
            code = debug_code_with_chatgpt(user_prompt, code, error_message, client)
            print('------------------')
            print("ChatGPT suggests the following fix:\n", code)
            print('------------------')

    # Save the last version of the code
    save_code_to_file(code)
    print("Final version saved to script.py")

if __name__ == "__main__":
    main()
1

Do you want to (1) enter a new code prompt or (2) modify existing code? Enter 1 or 2: 1
Please enter the code prompt: write an animation of the micro-canonical model
How many debug iterations should be done? 5
------------------
Initial code generated by ChatGPT:
 import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Model parameters
n = 100
temp = 2.0
state = np.ones((n, n))

def energy(state):
    return -np.sum(state * (np.roll(state, 1, 0) + np.roll(state, -1, 0) + np.roll(state, 1, 1) + np.roll(state, -1, 1)))

def step(state):
    for _ in range(n * n):
        x, y = np.random.randint(0, n, 2)
        flip = state.copy()
        flip[x, y] *= -1
        if np.random.rand() < np.exp(- (energy(flip) - energy(state)) / temp):
            state = flip
    return state

fig = plt.figure()

def animate(i):
    global state
    state = step(state)
    plt.imshow(state, cmap='binary', interpolation='nearest')

ani = animation.FuncAnimation(fig, a

<Figure size 640x480 with 0 Axes>

Code executed successfully.
Saved to script.py
Final version saved to script.py


/home/galen/anaconda3/lib/python3.11/site-packages/matplotlib/animation.py:884: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


1